# Vectorized and harmonized geological maps
![](./Charm-50.png)

## Pre-requeries

### GeoPandas installation

In [ ]:
!pip install geopandas

### Download and unzip dataset

In [ ]:
### http://infoterre.brgm.fr/telechargements/BDCharm50/GEO050K_HARM_001.zip
from glob import glob
import os

data_folder = '/tmp'
shapefile_paths = glob(os.path.join(data_folder, '*.shp'))

## Load and visualize dataset

### Load shape data as streamlines

In [ ]:
from bioexplorer import BioExplorer
import geopandas as gpd
from tqdm import tqdm
import seaborn as sns

be = BioExplorer()
core = be.core_api()
status = be.reset_scene()

DEFAULT_RADIUS = 25.0

errors = list()
palette = sns.color_palette('Blues', len(shapefile_paths))
i = 0
for shapefile_path in tqdm(shapefile_paths):
    c = palette[i]
    data = gpd.read_file(shapefile_path)
    line_geometry = data.geometry
    colors = list()
    indices = list()
    indices.append(0)
    vertices = list()
    index = 0
    for line in line_geometry:
        try:
            coordinates = line.coords
            if len(coordinates) <= 2:
                continue
            for coordinate in coordinates:
                vertices.append(coordinate[0])
                vertices.append(coordinate[1])
                vertices.append(0.0) # 2D dataset
                vertices.append(DEFAULT_RADIUS)
                colors.append(c[0])
                colors.append(c[1])
                colors.append(c[2])
                colors.append(1)
            index += len(coordinates)
            indices.append(index)
        except Exception as e:
            errors.append(e)
    if len(indices)>2:
        be.add_streamlines(os.path.basename(shapefile_path), indices, vertices, colors)
    i += 1

status = be.reset_camera()